# Laboratorium ćwiczenie 3 - Programowanie w obliczeniach inteligentnych
Autor: Aleksander Kretek 259130

1. Celem tego ćwiczenia było wykonanie zdjęć różnych rodzajów jednorodnych powierzchni.Następnie napisanie algorytmu służącego do wczytywanie ów zdjęć i wycinania z nich próbek(wymiary: 128x128).
2. Wycięte próbki następnie zapisać w odpowiednich katalogach.
3. Kolejnym krokiem było napisanie algorytmu do wczytywania próbek i wyznaczania dla nich cech tekstur na podstawie modelu macierzy zdarzeń.
Należało wyznaczyć następujące cechy:
a. dissimilarity, correlation, contrast, energy, homogeneity, ASM.
b. Przyjąć 3 odległości pikseli: 1, 3, 5
c. oraz 4 kierunki: 0, 45, 90 I 135 stopni (zakładamy symetrię kierunków).
d. Każdy wektor cech uzupełnić o nazwę kategorii tekstury
4. Następnie zapisać zbiór wektorów danych do pliku csv.
5. Ostatnim krokiem było napisanie skryptu służącego klasyfikacji wektorów cech z wykorzystaniem dowolnego algorytmu do klasyfikacji danych. Uczenie należało przeprowadzić dla wyodrębnionego zbioru treningowego, a testowanie dla zbioru testowego.

# Biblioteki

In [ ]:
import os
import numpy as np
import cv2
from skimage import io, color
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import glob

# Konfiguracja

1. INPUT_IMAGE_DIR należy wpisać ścieżkę do katalogu, w którym znajdują się podkatalogi ze zdjęciami.
2. PATCHES_OUTPUT_DIR należy wpisać ścieżkę do katalogu, w którym zapisane zostaną próbki.
3. FEATURES_CSV_FILE należy wpisać ścieżkę do pliku, w którym zapisane zostaną cechy próbek.

In [ ]:
INPUT_IMAGE_DIR = '/content/sample_data/zdjecia'  # Obrazy oryginalne
PATCHES_OUTPUT_DIR = '/content/sample_data/wycinki'   # Katalog, gdzie zapisane zostaną wycięte próbki
FEATURES_CSV_FILE = '/content/sample_data/features.csv' # Plik CSV do zapisu cech
PATCH_SIZE = (128, 128)                 # Rozmiar wycinanych próbek (wysokość, szerokość)
GLCM_DISTANCES = [1, 3, 5]              # Odległości dla GLCM
GLCM_ANGLES = [0, np.pi/4, np.pi/2, 3*np.pi/4] # Kąty dla GLCM (0, 45, 90, 135 stopni)
GLCM_LEVELS = 64                        # Liczba poziomów szarości

# Wczytywanie obrazów i wycinanie próbek

1. Funkcja przeszukuje wszystkie podkatalogi w katalogu input_dir.
2. Wycina próbki o określonym rozmiarze (patch_h, patch_w) z każdego obrazu.
3. Zapisuje próbki w odpowiednich podkatalogach w katalogu output_dir.
4. Każda próbka jest zapisywana z unikalną nazwą opartą na oryginalnej nazwie pliku i położeniu wycinka (r, c).

In [ ]:
def extract_patches(input_dir, output_dir, patch_h, patch_w):
    # Tworzenie katalogów na wycinki o ile nie istnieją
    os.makedirs(output_dir, exist_ok=True)
    categories = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    total_patches_saved = 0
    # Przetwarzanie każdej kategorii obrazów
    for category in categories:
        category_input_path = os.path.join(input_dir, category)
        category_output_path = os.path.join(output_dir, category)
        os.makedirs(category_output_path, exist_ok=True)
        # Wyszukuje wszystkie pliki graficzne w formatach
        image_files = glob.glob(os.path.join(category_input_path, '*.png')) + \
                      glob.glob(os.path.join(category_input_path, '*.jpg')) + \
                      glob.glob(os.path.join(category_input_path, '*.jpeg')) + \
                      glob.glob(os.path.join(category_input_path, '*.bmp')) + \
                      glob.glob(os.path.join(category_input_path, '*.tif'))

        category_patches_count = 0
        # Wczytywanie obrazu, pobieranie jego wymiarów, pobiera nazwe pliku
        for img_path in image_files:
            try:
                img = cv2.imread(img_path)

                img_h, img_w = img.shape[:2]
                img_filename = os.path.basename(img_path)
                img_name_part = os.path.splitext(img_filename)[0]
                # Wycinanie próbek
                for r in range(0, img_h - patch_h + 1, patch_h):
                    for c in range(0, img_w - patch_w + 1, patch_w):
                        patch = img[r:r + patch_h, c:c + patch_w]

                        # Zapisywanie próbki
                        patch_filename = f"{img_name_part}_patch_{r:04d}_{c:04d}.png"
                        patch_output_path = os.path.join(category_output_path, patch_filename)
                        cv2.imwrite(patch_output_path, patch)
                        category_patches_count += 1

            except Exception as e:
                print(f"BŁĄD podczas przetwarzania obrazu {img_path}: {e}")

        print(f"Zapisano {category_patches_count} próbek dla kategorii '{category}'.")
        total_patches_saved += category_patches_count

# Wczytywanie próbek i obliczanie cech GLCM

Funkcja calculate_glcm_features przetwarza wycięte próbki tekstur:

1. Wczytuje każdą próbkę z podkatalogów w patches_dir.

2. Przekształca obraz do skali szarości.

3. Zmniejsza głębię jasności do levels.

4. Oblicza macierz GLCM dla zadanych odległości (distances) i kątów (angles). Macierz jest symetryczna i normalizowana.

5. Z macierzy GLCM wyznacza następujące cechy: dissimilarity, correlation, contrast, energy, homogeneity, ASM. Dla każdej z tych cech obliczana jest średnia wartość dla wszystkich kierunków (kątów) przy danej odległości.

6. Każdy wektor cech jest uzupełniany o nazwę kategorii tekstury oraz nazwę pliku próbki.

7. Zwraca listę słowników, gdzie każdy słownik reprezentuje wektor cech jednej próbki.

In [ ]:
def calculate_glcm_features(patches_dir, distances, angles, levels):
    # Pobieranie listy podkatalogów z katalogu, w którym znajdują się wycinki
    categories = [d for d in os.listdir(patches_dir) if os.path.isdir(os.path.join(patches_dir, d))]

    all_features = []
    properties = ['dissimilarity', 'correlation', 'contrast', 'energy', 'homogeneity', 'ASM']
    # Iteracja po każdym podkatalogu
    for category in categories:
        print(f"Przetwarzanie kategorii: {category}")
        category_path = os.path.join(patches_dir, category)
        patch_files = glob.glob(os.path.join(category_path, '*.png'))

        processed_patches = 0
        # Iteracja po każdej próbce
        for patch_path in patch_files:
            try:
                patch = io.imread(patch_path)
                # Konwersja do skali szarości
                if patch.ndim == 3:
                    patch_gray = color.rgb2gray(patch)
                elif patch.ndim == 2:
                    patch_gray = patch
                else:
                    continue
                if patch_gray.max() <= 1.0:
                     patch_gray = (patch_gray * 255).astype(np.uint8)
                else:
                     patch_gray = patch_gray.astype(np.uint8)
                img_quantized = np.floor(patch_gray / 256. * levels).astype(np.uint8)

                # Obliczenie macierzy GLCM
                glcm = graycomatrix(img_quantized,
                                    distances=distances,
                                    angles=angles,
                                    levels=levels,
                                    symmetric=True,
                                    normed=True)

                # Obliczenie cech GLCM
                feature_vector = {'category': category, 'patch_file': os.path.basename(patch_path)}
                for prop in properties:
                    prop_values = graycoprops(glcm, prop)
                    for i, dist in enumerate(distances):
                        feature_vector[f'{prop}_d{dist}'] = np.mean(prop_values[i, :])
                # Dodanie cech do listy "all_features"
                all_features.append(feature_vector)
                processed_patches += 1

            except Exception as e:
                print(f"BŁĄD podczas przetwarzania próbki {patch_path}: {e}")

    return all_features

# Zapis wektorów cech do pliku CSV
1. Funkcja save_features_to_csv wykorzystuje bibliotekę Pandas do zapisania listy wektorów cech (otrzymanej z calculate_glcm_features) do pliku CSV.

In [ ]:
def save_features_to_csv(features_list, csv_filepath):
    try:
        df = pd.DataFrame(features_list)
        if 'category' in df.columns:
             cols = ['category'] + [col for col in df.columns if col != 'category']
             df = df[cols]
        df.to_csv(csv_filepath, index=False)
        print(f"Zapisano {len(df)} wektorów cech do pliku: {csv_filepath}")
    except Exception as e:
        print(f"BŁĄD podczas zapisywania pliku CSV {csv_filepath}: {e}")

# Klasyfikacja wektorów cech

Funkcja classify_features realizuje proces klasyfikacji:
1. Wczytuje dane z pliku CSV.
2. Przygotowuje dane:
- X: wektory cech (usuwając kolumny category i patch_file).
- y: etykiety kategorii.
  
3. Koduje etykiety tekstowe na wartości liczbowe za pomocą LabelEncoder.
4. Dzieli dane na zbiór treningowy i testowy (train_test_split) z zachowaniem proporcji klas (stratify=y).
5. Wybiera, trenuje i testuje klasyfikator:
- K-Najbliższych Sąsiadów (KNeighborsClassifier).
- Maszyna Wektorów Nośnych (SVC).
  
6. Oblicza i wyświetla dokładność (accuracy_score) klasyfikatora na zbiorze testowym.

In [ ]:
def classify_features(csv_filepath, test_size=0, random_state=0, classifier_type=0):

    try:
        # Wczytywanie danych
        df = pd.read_csv(csv_filepath)
        print(f"Wczytano {len(df)} wektorów cech z {csv_filepath}.")

        if df.empty:
            print("BŁĄD: Plik CSV jest pusty.")
            return
        if 'category' not in df.columns:
            print("BŁĄD: Brak kolumny 'category' w pliku CSV.")
            return

        # Przygotowanie danych
        X = df.drop(['category', 'patch_file'], axis=1) # Cechy
        y_raw = df['category']                     # Etykiety

        # Kodowanie etykiet (zamiana nazw kategorii na liczby)
        le = LabelEncoder()
        y = le.fit_transform(y_raw)

        # Podział na zbiór treningowy i testowy
        X_train, X_test, y_train, y_test = train_test_split(X, y,
            test_size=test_size,
            random_state=random_state,
            stratify=y
        )
        print(f"Podzielono dane na zbiór treningowy ({len(X_train)} próbek) i testowy ({len(X_test)} próbek).")

        # Wybór i trenowanie klasyfikatora
        if classifier_type.lower() == 'knn':
            print("(KNN):")
            model = KNeighborsClassifier(n_neighbors=5)
        elif classifier_type.lower() == 'svm':
            print("(SVM):")
            model = SVC(kernel='linear',  random_state=random_state)
        # Trenowanie modelu
        model.fit(X_train, y_train)
        # Predykcja na zbiorze testowym
        y_pred = model.predict(X_test)
        # Ocena dokładności modelu
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Dokładność (Accuracy) na zbiorze testowym: {accuracy:.4f} ({accuracy*100:.2f}%)")

    except Exception as e:
        print(f"BŁĄD podczas klasyfikacji: {e}")

# Wywołanie funkcji

In [ ]:
if __name__ == "__main__":
    extract_patches(INPUT_IMAGE_DIR, PATCHES_OUTPUT_DIR, PATCH_SIZE[0], PATCH_SIZE[1])
    features = calculate_glcm_features(PATCHES_OUTPUT_DIR, GLCM_DISTANCES, GLCM_ANGLES, GLCM_LEVELS)
    save_features_to_csv(features, FEATURES_CSV_FILE)
    classify_features(FEATURES_CSV_FILE, test_size=0.90, random_state=42, classifier_type='svm')
    classify_features(FEATURES_CSV_FILE, test_size=0.2, random_state=42, classifier_type='knn')

Zapisano 40 próbek dla kategorii 'tynk'.
Zapisano 25 próbek dla kategorii 'laminat'.
Zapisano 0 próbek dla kategorii '.ipynb_checkpoints'.
Zapisano 16 próbek dla kategorii 'gres'.
Zapisano 9 próbek dla kategorii 'carbon'.
Przetwarzanie kategorii: tynk
Przetwarzanie kategorii: laminat
Przetwarzanie kategorii: .ipynb_checkpoints
Przetwarzanie kategorii: gres
Przetwarzanie kategorii: carbon
Zapisano 90 wektorów cech do pliku: /content/sample_data/features.csv
Wczytano 90 wektorów cech z /content/sample_data/features.csv.
Podzielono dane na zbiór treningowy (9 próbek) i testowy (81 próbek).
(SVM):
Dokładność (Accuracy) na zbiorze testowym: 0.8025 (80.25%)
Wczytano 90 wektorów cech z /content/sample_data/features.csv.
Podzielono dane na zbiór treningowy (72 próbek) i testowy (18 próbek).
(KNN):
Dokładność (Accuracy) na zbiorze testowym: 0.8889 (88.89%)
